# Fake News Prediction using ML

### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# print those stopwords
stopwords = stopwords.words('english')
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [4]:
len(stopwords)

179

## Data Preprocessing

In [55]:
news_data = pd.read_csv('fake_train.csv')

news_data.head(5)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


#### Classify News in - Real news or Fake news
#### 0 means  - Real news
#### 1 means  - Fake News

In [6]:
news_data.shape

(20800, 5)

In [7]:
# Check null values in data

news_data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
558+1957+39

2554

In [9]:
# Replce null values with null string

news_data = news_data.fillna('')

In [10]:
news_data.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [11]:
# Merging news title and author name

news_data['content'] = news_data['author']+" "+news_data['title']

In [12]:
news_data['content']

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object

### Stemming of words : - process of reducing words to its root word.    
#### eg. actor, actors, acting => act

In [13]:
port_stem = PorterStemmer()

In [14]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [15]:
news_data['content'] = news_data['content'].apply(stemming)

In [16]:
print(news_data['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [17]:
# Seperate data  and label

X = news_data['content'].values
y = news_data['label'].values

In [18]:
X

array(['darrel lucu hous dem aid even see comey letter jason chaffetz tweet',
       'daniel j flynn flynn hillari clinton big woman campu breitbart',
       'consortiumnew com truth might get fire', ...,
       'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time',
       'alex ansari nato russia hold parallel exercis balkan',
       'david swanson keep f aliv'], dtype=object)

In [19]:
y

array([1, 0, 1, ..., 0, 1, 1], dtype=int64)

In [20]:
y.shape

(20800,)

In [21]:
# Converting text data to numeric data

vectorizer = TfidfVectorizer()

In [22]:
X = vectorizer.fit_transform(X)

In [27]:
print(X)

  (0, 15686)	0.28485063562728646
  (0, 2483)	0.3676519686797209
  (0, 7692)	0.24785219520671603
  (0, 8630)	0.29212514087043684
  (0, 2959)	0.2468450128533713
  (0, 13473)	0.2565896679337957
  (0, 4973)	0.233316966909351
  (0, 267)	0.27010124977708766
  (0, 3792)	0.2705332480845492
  (0, 7005)	0.21874169089359144
  (0, 8909)	0.3635963806326075
  (0, 3600)	0.3598939188262559
  (1, 1894)	0.15521974226349364
  (1, 2223)	0.3827320386859759
  (1, 16799)	0.30071745655510157
  (1, 1497)	0.2939891562094648
  (1, 2813)	0.19094574062359204
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (2, 5389)	0.3866530551182615
  (2, 5968)	0.3474613386728292
  (2, 9620)	0.49351492943649944
  (2, 15611)	0.41544962664721613
  (2, 2943)	0.3179886800654691
  :	:
  (20797, 1287)	0.3353805680413986
  (20797, 13122)	0.24825263521976057
  (20797, 12344)	0.27263457663336677
  (20797, 14967)	0.3115945315488075
  (20797, 12138)	0.24778257724396505
  (20797, 9518)	0.295420

#### Train test Split

In [32]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, stratify=y, random_state=2)

In [33]:
print("X shape : ", X.shape)
print("Xtrain shape : ", Xtrain.shape)
print("Xtest shape : ", Xtest.shape)
print("ytrain shape : ", ytrain.shape)
print("ytest shape : ", ytest.shape)

X shape :  (20800, 17128)
Xtrain shape :  (16640, 17128)
Xtest shape :  (4160, 17128)
ytrain shape :  (16640,)
ytest shape :  (4160,)


## Model Training

In [34]:
model = LogisticRegression()

In [35]:
model.fit(Xtrain, ytrain)

LogisticRegression()

### Model Evaluation

#### Accuracy Score

In [38]:
# Accuracy on training data 

Xtrain_pred = model.predict(Xtrain)
train_accuracy = accuracy_score(Xtrain_pred, ytrain)

In [39]:
print("Acuracy of training data is : ", train_accuracy)

Acuracy of training data is :  0.9865985576923076


In [40]:
# Accuracy on training data 

Xtest_pred = model.predict(Xtest)
test_accuracy = accuracy_score(Xtest_pred, ytest)

In [41]:
print("Acuracy of test data is : ", test_accuracy)

Acuracy of test data is :  0.9790865384615385


In [46]:
print("Acuracy of training data is : ", round(train_accuracy*100, 2))
print("Acuracy of test data is : ", round(test_accuracy*100, 2))

Acuracy of training data is :  98.66
Acuracy of test data is :  97.91


## Bulding a predictive system

#### 1st Case

In [47]:
new_news =  Xtest[0]       # first news of Xtest data

In [49]:
prediction = model.predict(new_news)

print(prediction)

if (prediction[0]==0):
    print("The News is Real")
    
else:
    print("The News is Fake")

[1]
The News is Fake


In [50]:
# Check with ytest 

print(ytest[0])

1


#### 2nd Case

In [51]:
new_news_1 =  Xtest[5] 

prediction_1 = model.predict(new_news)

print(prediction_1)

if (prediction_1[0]==0):
    print("The News is Real")
    
else:
    print("The News is Fake")

[1]
The News is Fake


In [52]:
# Check with ytest also

print(ytest[5])

1


#### We can say that our model predicted correctly in both cases